In [1]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain
import openai

class configs():
    openai_api_key= "your openai api-key"
    openai.api_key =openai_api_key
    openai.api_base="https://api.closeai-proxy.xyz/v1"
    # path to the reference paper that describes the semantic communications
    pdf_path = "1.pdf"
    # path to the reference code that is as an example
    code_path = "code.txt"
    # iteration number
    iterations = 3

In [28]:
# Define LLM-enhanced multi-agent system
class LLM_enhanced_multi_agent_system_for_SC():
    def __init__(self, args:configs):
        self.api_key = args.openai_api_key
        self.llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.9, openai_api_key=self.api_key)
        self.iterations = args.iterations
        self.embeddings = OpenAIEmbeddings(openai_api_key=self.api_key)

    # Validate the legitimacy of the input
    def secure_agent(self,query):
        response = openai.Moderation.create(
        input=query,
        )
        moderation_output = response["results"][0]
        print(moderation_output)
        if moderation_output["flagged"]:
            print("Illegal input!")
            return False
        else:
            return query

    # Load the paper and generate vector memory
    def condensate_agent(self, pdf_path=""):
        loader = PyPDFLoader(pdf_path)
        docs = loader.load()
        self.db = DocArrayInMemorySearch.from_documents(
            docs,
            self.embeddings
        )
    # Distill relevant SC knowledge for constructing the SC model
    def inference_agent(self,query):
        retriever = self.db.as_retriever()
        qa_stuff = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",# Stuff means merging text fragments into one paragraph of text
            retriever=retriever,
            verbose=True
        )
        response = qa_stuff.run(query)
        display(Markdown(response))
        return response

    # Formulates a specific sub-task chain for the SC model
    def planning_agent(self, input_name="paper_ref", output_name="sc_scheme"):
        scheme_prompt = ChatPromptTemplate.from_template(
            "# Referring to the following description of the semantic communication model: '''{"
            f"{input_name}"
            "}'''."
            "# Output the design scheme for achieving the semantic communication model."
        )
        chain = LLMChain(llm=self.llm, prompt=scheme_prompt,
                             output_key=output_name,
                             )
        return chain
    # sub-task chain for code generation
    def code_generation(self,input_name="sc_scheme", output_name="sc_model"):
        data = [
            "Hello there!",
            "How are you doing?",
            "PyTorch is great.",
            "Machine learning is fascinating.",
            "Let's build something amazing.",
            "Semantic communication matters.",
            "Understanding context is important.",
            "AI is changing the world.",
            "Keep learning and growing.",
            "Innovation drives progress."
        ]
        model_prompt = ChatPromptTemplate.from_template(
            "# According to the design scheme of the semantic communication model: '''{"
            f"{input_name}"
            "}'''."
            "# Refering to the reference code: '''{ref_code}''',"
            "output the Pytorch framework-based Python code for achieving the semantic communication model, "
            f"assuming the input data is '''{data}''', which is a list of sentences. "
            f"Specifically, the Additive Gaussian White Noise (AWGN) can serve as the physical channel. The Bilingual Evaluation Understudy (BLEU) score is adopted as the metric that evaluates the SC model. We expect that the generated SC model achieves no less than a 0.6 BLEU score when the Signal-to-Noise Ratio (SNR) is 10 dB. The total number of model parameters does not exceed 2,000,000 for the resource constraints of devices."
        )
        chain = LLMChain(llm=self.llm, prompt=model_prompt,
                             output_key=output_name,
                             )
        return chain

    # evaluate the quality of the generated code
    def evaluate_agent(self, input_name="sc_model", output_name="eval_score"):
        # Manually input bleu scores for SC model
        eval_prompt = ChatPromptTemplate.from_template(
            "The quality of the generated code is evaluated and an overall score is given from three aspects: "
            "1. Quality of code, including its reasonableness and completeness."
            "2. The performance of code, using bleu score as the evaluation indicator."
            "3. Whether the number of parameters in the generated code meets the limitation."
            "Based on the above requirements, evaluate the quality of the 'generated code':'''{"
            f"{input_name}"
            "}'''."
            "## Output the evaluated results. The format of the output refers to:"
            "```json Dict('model parameters': int \ the model parameters of the 'generated code', 'evaluated score': int \ The evaluated score is based on the performance of the reference code in three aspects. The evaluated score ranges from 0 to 100 and is not the same as the last time."
            "'evaluation': string \ Based on the performance of the reference code in three aspects, give the reviews, including the model parameters, the description of 'generated code', the advantage and disadvantage analysis of the 'generated code', etc."
            "There is an example: 'The Python-based SC model has been successfully implemented, incorporating all necessary modules. The semantic encoder and decoder are realized using Long Short-Term Memory (LSTM) networks. The channel encoder and decoder are constructed based on the Multilayer Perceptron (MLP) architecture.  The final SC model can achieve a 0.68 BLEU score when SNR is 10 dB, which meets expectations. In addition, the total number of model parameters is 1,826,762.')```"
            )
        chain = LLMChain(llm=self.llm, prompt=eval_prompt,
                                 output_key=f"{output_name}",
                                 )
        return chain

    # Improve the generated codes according to suggestion
    def reflexion_and_refinement_agents(self,inputs, output_name="sc_model"):
        model_prompt = ChatPromptTemplate.from_template(
        "# Modify the Python code: '''{"
        f"{inputs[0]}"
        "}'''."
        "# According to the corresponding evaluation results:'''{"
        f"{inputs[1]}"
        "}'''."
        "# If the 'evaluated score'< 60, a major revision is required (e.g., change the architecture of the networks); otherwise, it is considered a minor revision (e.g., change the layers of the networks)."
        "# Output the modified Python codes that aim to improve the 'evaluated score' and obtain a score of no less than 90 finally."
        )
        chain = LLMChain(llm=self.llm, prompt=model_prompt,
                             output_key=output_name,
                             )
        return chain

    # Combine all agents for SC system generation
    def create_chains(self):
        chains = []
        output_keys = []
        # define sc scheme 1
        input_name = "paper_ref"
        output_name = "scheme_1"
        chain = self.planning_agent(input_name,output_name)
        chains.append(chain)
        output_keys.append(output_name)
        # define sc scheme 2
        input_name = "paper_ref"
        output_name = "scheme_2"
        chain = self.planning_agent(input_name, output_name)
        chains.append(chain)
        output_keys.append(output_name)
        # define code generation 1
        input_name = "scheme_1"
        output_name = "sc_model_0_1"
        chain = self.code_generation(input_name,output_name)
        chains.append(chain)
        output_keys.append(chain.output_key)

        # define code generation 2
        input_name = "scheme_2"
        output_name = "sc_model_0_2"
        chain = self.code_generation(input_name,output_name)
        chains.append(chain)
        output_keys.append(chain.output_key)
        for i in range(self.iterations):
            # eval sc model
            input_name = f"sc_model_{i}_1"
            output_name = f"eval_score_{i}_1"
            chain = self.evaluate_agent(input_name,output_name)
            chains.append(chain)
            output_keys.append(chain.output_key)

            input_name = f"sc_model_{i}_2"
            output_name = f"eval_score_{i}_2"
            chain = self.evaluate_agent(input_name,output_name)
            chains.append(chain)
            output_keys.append(chain.output_key)

            # improve sc model
            inputs = [f"sc_model_{i}_1",f"eval_score_{i}_1"]
            output_name = f"sc_model_{i+1}_1"
            chain = self.reflexion_and_refinement_agents(inputs,output_name)
            chains.append(chain)
            output_keys.append(chain.output_key)

            inputs = [f"sc_model_{i}_2",f"eval_score_{i}_2"]
            output_name = f"sc_model_{i+1}_2"
            chain = self.reflexion_and_refinement_agents(inputs,output_name)
            chains.append(chain)
            output_keys.append(chain.output_key)

        # eval the final sc model
        input_name = f"sc_model_{self.iterations}_1"
        output_name = f"eval_score_{self.iterations}_1"
        chain = self.evaluate_agent(input_name,output_name)
        chains.append(chain)
        output_keys.append(chain.output_key)

        input_name = f"sc_model_{self.iterations}_2"
        output_name = f"eval_score_{self.iterations}_2"
        chain = self.evaluate_agent(input_name,output_name)
        chains.append(chain)
        output_keys.append(chain.output_key)

        self.overall_chain = SequentialChain(
            chains=chains,
            input_variables=["paper_ref","ref_code"],
            output_variables=output_keys,
            verbose=True,
        )

    # Run multi-agent system
    def Run(self, inputs):
        outputs = self.overall_chain(inputs)
        return outputs

In [29]:
# generate sc by LLM enhanced multi-agent system
args = configs()
ref_code = open(args.code_path,"r",encoding="utf-8").read()
LL_SC = LLM_enhanced_multi_agent_system_for_SC(args)
LL_SC.condensate_agent(args.pdf_path)
paper_query = "Composition of the semantic communication model"
if LL_SC.secure_agent(paper_query):
    SC_Components = LL_SC.inference_agent(paper_query)
    LL_SC.create_chains()
    results = LL_SC.Run({"paper_ref":SC_Components,"ref_code":ref_code})
else:
    pass

{
  "categories": {
    "harassment": false,
    "harassment/threatening": false,
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "self-harm/instructions": false,
    "self-harm/intent": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": false,
    "violence/graphic": false
  },
  "category_scores": {
    "harassment": 0.00019458516908343881,
    "harassment/threatening": 6.477488113887375e-06,
    "hate": 0.003069685073569417,
    "hate/threatening": 5.001786121283658e-06,
    "self-harm": 7.029921562207164e-06,
    "self-harm/instructions": 2.7231806143390713e-06,
    "self-harm/intent": 9.790189778868807e-07,
    "sexual": 0.0002446114958729595,
    "sexual/minors": 3.861287405015901e-05,
    "violence": 0.00010841531184269115,
    "violence/graphic": 5.488210081239231e-05
  },
  "flagged": false
}


> Entering new RetrievalQA chain...

> Finished chain.


The semantic communication model consists of the following components:
1. Semantic Encoder: Extracts semantic information from the original data and encodes it into semantic features.
2. Channel Encoder: Encodes and modulates the semantic features for transmission over the physical channel.
3. Channel Decoder: De-modulates and decodes the received signal to recover the transmitted semantic features.
4. Semantic Decoder: Understands the received semantic features, infers semantic information, and recovers the original data.
5. Knowledge Base: A universal knowledge model that helps the semantic encoder and decoder understand and infer semantic information effectively.



> Entering new SequentialChain chain...

> Finished chain.


In [30]:
results['paper_ref']

'The semantic communication model consists of the following components:\n1. Semantic Encoder: Extracts semantic information from the original data and encodes it into semantic features.\n2. Channel Encoder: Encodes and modulates the semantic features for transmission over the physical channel.\n3. Channel Decoder: De-modulates and decodes the received signal to recover the transmitted semantic features.\n4. Semantic Decoder: Understands the received semantic features, infers semantic information, and recovers the original data.\n5. Knowledge Base: A universal knowledge model that helps the semantic encoder and decoder understand and infer semantic information effectively.'

In [44]:
results['sc_model_3_2']

'```python\nimport torch\nimport torch.nn as nn\nimport torch.optim as optim\nimport numpy as np\nimport math\n\n# Generate random sentences as data A\ndata_A = [\n    "Hello there!",\n    "How are you doing?",\n    "PyTorch is great.",\n    "Machine learning is fascinating.",\n    "Let\'s build something amazing.",\n    "Semantic communication matters.",\n    "Understanding context is important.",\n    "AI is changing the world.",\n    "Keep learning and growing.",\n    "Innovation drives progress."\n]\n\n# Tokenize sentences into words\nwords = [sentence.split() for sentence in data_A]\n\n# Create a vocabulary\nvocab = list(set(word for sentence in words for word in sentence))\nvocab_size = len(vocab)\n\n# Convert words to unique indices\nword_to_idx = {word: idx for idx, word in enumerate(vocab)}\nidx_to_word = {idx: word for word, idx in word_to_idx.items()}\n\n# Convert sentences to numerical representation\nnumerical_sentences = [[word_to_idx[word] for word in sentence] for sente

In [40]:
results['eval_score_3_2']

'```json\n{\n    "model parameters": 457042,\n    "evaluated score": 75,\n    "evaluation": "The Python-based semantic communication model has been implemented with necessary components including Semantic Encoder, Channel Encoder, Channel Decoder, Semantic Decoder, and Physical Channel modules. The model is trained using sentences converted into numerical representations and optimized using the Adam optimizer with CrossEntropyLoss. The BLEU score evaluation indicates reasonable performance in generating predicted sentences. The advantage of the generated code is the comprehensive implementation of the model components and training process. However, the disadvantage is the relatively high number of model parameters, exceeding the limitation set for this evaluation. Overall, the model demonstrates good quality and performance for semantic communication tasks."\n}\n```'